Brandon Connors  
10/1/23  
Web Scraping with Jupyter Notebooks

In [7]:
# These are the libraries I used for this project
#pip install urllib3
#pip install certifi
#pip install pandas
#pip install Jinja2

In [8]:
import urllib3
import certifi

# This is the sample code the professor posted on the discussion boards, I liked it because of the use of 'f' formatting for the link to insert the variables.
# My go to would have been to use concatenation, but I felt this was a bit more elegant and wanted to use it.
year = 2015
page = 1
link = f"https://www.metacritic.com/browse/movie/all/all/{year}/metascore/?page={page}"
http = urllib3.PoolManager(ca_certs=certifi.where())
response = http.request('GET', link, headers={'User-Agent': 'Mozilla/5.0'})
datastring = str(response.data, "utf-8")
print(f"Fetched {len(datastring)} characters from {link}: {response.status}")

Fetched 322075 characters from https://www.metacritic.com/browse/movie/all/all/2015/metascore/?page=1: 200


In [9]:
import re

# Here I am searching for the regex strings I created in the first assignment and assigning those to lists
movieTitles = re.findall('<div data-title=\"(.*?)\"', datastring)
releaseDates = re.findall('<span class=\"u-text-uppercase\">\n\s+(.*?)\n\s+<', datastring)
movieDescriptions = re.findall('description\"><span>(.*?)<\/span', datastring)
metacriticScores = re.findall('<span data-v-4cdca868>(.*?)<', datastring)
rawThumbnailURLs = re.findall('<img src=\"(.*?)\" height', datastring)

# Fixing the URLs so they're actually clickable
for i in range(len(rawThumbnailURLs)):
    thumbnailURLs = [sub.replace('amp;', '') for sub in rawThumbnailURLs]

# Sanity check here to make sure I caught everything
print('Titles:', len(movieTitles), 'Dates: ', len(releaseDates), 'Descriptions: ', len(movieDescriptions), 'Scores: ', len(metacriticScores), 'URLs: ', len(thumbnailURLs))

# print(movieTitles)
# print(releaseDates)
# print(movieDescriptions)
# print(metacriticScores)
# print(thumbnailURLs)

Titles: 24 Dates:  24 Descriptions:  24 Scores:  24 URLs:  24


In [10]:
# Initializing the main database list
database = []
# Iterating through each item the lists above to grab each corresponding item and putting them into a list together, then putting that movie's list into the database list
for i in range(len(movieTitles)):
    newAddition = []
    newAddition.append(movieTitles[i])
    newAddition.append(releaseDates[i])
    newAddition.append(metacriticScores[i])
    newAddition.append(thumbnailURLs[i])
    newAddition.append(movieDescriptions[i])
    database.append(newAddition)

In [15]:
import pandas as pd

# This sets the dataframe to not truncate the data, useful if you want to actually see the whole thing
pd.set_option('display.max_colwidth', None)

# This creates the dataframe and formats the resulting table so that everything is left aligned, making it easier to read.
# Alignment code I got from: https://www.geeksforgeeks.org/align-columns-to-left-in-pandas-python/
df = pd.DataFrame(database, columns=['Movie Title', 'Release Date', 'Metacritic Score', 'Thumbnail URL', 'Movie Description'])
left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
left_aligned_df = left_aligned_df.set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])])

display(left_aligned_df)

,Movie Title,Release Date,Metacritic Score,Thumbnail URL,Movie Description
0,45 Years,"Dec 23, 2015",94,https://www.metacritic.com/a/img/resize/6a5bd5bbb7974005d66c3fe569c21e3786ea3145/catalog/provider/2/2/2-7e75f9d82b9daa3efdb74ef6d7d5505f.jpg?auto=webp&fit=cover&height=132&width=88,"There is just one week until Kate Mercer's (Charlotte Rampling) 45th wedding anniversary and the planning for the party is going well. But then a letter arrives for her husband (Tom Courtenay). The body of his first love has been discovered, frozen and preserved in the icy glaciers of the Swiss Alps. By the time the party is upon them, five days later, there may not be a marriage left to celebrate. [IFC Films]"
1,Carol,"Nov 20, 2015",94,https://www.metacritic.com/a/img/resize/7b3783def087f9b94e7a0391870fc81d08cc8449/catalog/provider/2/2/2-f7316f7e14637d049239318771fdd1a0.jpg?auto=webp&fit=cover&height=132&width=88,"Set in 1950s New York, two women from very different backgrounds find themselves in the throes of love. A young woman in her 20s, Therese Belivet (Rooney Mara), is a clerk working in a Manhattan department store and dreaming of a more fulfilling life when she meets Carol (Cate Blanchett), an alluring woman trapped in a loveless, convenient marriage. As an immediate connection sparks between them, the innocence of their first encounter dims and their connection deepens. While Carol breaks free from the confines of marriage, her husband (Kyle Chandler) threatens her competence as a mother when Carol’s involvement with Therese and close relationship with her best friend Abby (Sarah Paulson) comes to light. As Carol leaves the comfort of home to travel with Therese, an internal journey of self-discovery coincides with her new sense of space. [TWC]"
2,Inside Out,"Jun 19, 2015",94,https://www.metacritic.com/a/img/resize/54c04aa6b209fdf9bf3aface7457b9bfdfd036fc/catalog/provider/2/2/2-ec41dc3dbeb08defaa20a5ff93a1fc64.jpg?auto=webp&fit=cover&height=132&width=88,"Growing up can be a bumpy road, and it’s no exception for Riley, who is uprooted from her Midwest life when her father starts a new job in San Francisco. Like all of us, Riley is guided by her emotions: Joy (Amy Poehler), Fear (Bill Hader), Anger (Lewis Black), Disgust (Mindy Kaling) and Sadness (Phyllis Smith). The emotions live in Headquarters, the control center inside Riley’s mind, where they help advise her through everyday life. As Riley and her emotions struggle to adjust to a new life in San Francisco, turmoil ensues in Headquarters. Although Joy, Riley’s main and most important emotion, tries to keep things positive, the emotions conflict on how best to navigate a new city, house, and school. [Pixar]"
3,Spotlight,"Nov 6, 2015",93,https://www.metacritic.com/a/img/resize/35758d6a35b5992b36081c441e9dd3869b461a25/catalog/provider/2/2/2-337ed9394803aa3e504a12ae7abf5eaf.jpg?auto=webp&fit=cover&height=132&width=88,"Spotlight tells the riveting true story of the Pulitzer Prize-winning Boston Globe investigation that would rock the city and cause a crisis in one of the world’s oldest and most trusted institutions. When the newspaper’s tenacious “Spotlight” team of reporters delve into allegations of abuse in the Catholic Church, their year-long investigation uncovers a decades-long cover-up at the highest levels of Boston's religious, legal, and government establishment, touching off a wave of revelations around the world. [Open Road Films]"
4,Sherpa,"Oct 2, 2015",93,https://www.metacritic.com/a/img/resize/ffb78b3e2560281c977d60448ad5b62afc566d14/catalog/provider/2/2/2-83789aa521d7a7a10473bb311623ec41.jpg?auto=webp&fit=cover&height=132&width=88,"A fight on Everest? It seemed incredible. But in 2013 news channels around the world reported an ugly brawl at 21,000ft as European climbers fled a mob of angry Sherpas. In 1953, New Zealander Edmund Hillary and Sherpa Tenzing Norgay had reached the summit in a spirit of co-operation and brave optimism. Now climbers and Sherpas were trading insu